In [3]:
import torch
import os
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from tqdm.notebook import tqdm  # Per vedere la barra di progresso del filtro

class CocoFasterRCNNDataset(Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        
        # --- MODIFICA FONDAMENTALE QUI ---
        # Invece di prendere tutti gli ID dal JSON alla cieca:
        # self.ids = list(sorted(self.coco.imgs.keys()))
        
        # Facciamo un controllo preventivo:
        all_ids = list(sorted(self.coco.imgs.keys()))
        self.ids = []
        
        print("Verifica delle immagini presenti su disco in corso...")
        # Scansioniamo gli ID e teniamo solo quelli che hanno un file corrispondente
        for img_id in tqdm(all_ids):
            path = self.coco.loadImgs(img_id)[0]['file_name']
            full_path = os.path.join(self.root, path)
            
            if os.path.exists(full_path):
                self.ids.append(img_id)
        
        print(f"Fatto! Trovate {len(self.ids)} immagini su disco (rispetto alle {len(all_ids)} nel JSON).")
        # ----------------------------------

    def __getitem__(self, index):
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        coco_annotation = coco.loadAnns(ann_ids)
        
        path = coco.loadImgs(img_id)[0]['file_name']
        # Qui ora siamo sicuri che il file esiste!
        img = Image.open(os.path.join(self.root, path)).convert("RGB")

        num_objs = len(coco_annotation)
        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            w = coco_annotation[i]['bbox'][2]
            h = coco_annotation[i]['bbox'][3]
            
            xmax = xmin + w
            ymax = ymin + h
            
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(coco_annotation[i]['category_id'])
            areas.append(coco_annotation[i]['area'])
            iscrowd.append(coco_annotation[i]['iscrowd'])

        # Gestione immagini senza box (background only)
        if num_objs > 0:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labels, dtype=torch.int64)
            areas = torch.as_tensor(areas, dtype=torch.float32)
            iscrowd = torch.as_tensor(iscrowd, dtype=torch.int64)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
            areas = torch.zeros((0,), dtype=torch.float32)
            iscrowd = torch.zeros((0,), dtype=torch.int64)

        image_id = torch.tensor([img_id])

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = areas
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.ids)

def collate_fn(batch):
    return tuple(zip(*batch))


In [4]:
# Percorsi (aggiornali se necessario)
IMG_DIR = '/kaggle/input/coco-reduced/kaggle/working/train'
ANN_FILE = '/kaggle/input/coco-2017-dataset/coco2017/annotations/instances_train2017.json'

# Trasformazione base
def get_transform():
    return T.Compose([T.ToTensor()])

# Istanziamo il Dataset
dataset = CocoFasterRCNNDataset(
    root=IMG_DIR,
    annotation=ANN_FILE,
    transforms=get_transform()
)

# Creiamo il DataLoader
data_loader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)

print("✅ Dataset e DataLoader creati con successo (Custom Class).")

# --- VERIFICA FUNZIONAMENTO ---
# Estraiamo un batch per essere sicuri al 100%
try:
    images, targets = next(iter(data_loader))
    print(f"\nBatch caricato!")
    print(f"Numero immagini: {len(images)}")
    print(f"Shape prima immagine: {images[0].shape}")
    print(f"Target keys: {targets[0].keys()}")
    print(f"Box format (primo box): {targets[0]['boxes'][0] if len(targets[0]['boxes']) > 0 else 'Nessun box'}")
except Exception as e:
    print(f"❌ Errore: {e}")


loading annotations into memory...
Done (t=20.65s)
creating index...
index created!
Verifica delle immagini presenti su disco in corso...


  0%|          | 0/118287 [00:00<?, ?it/s]

Fatto! Trovate 11829 immagini su disco (rispetto alle 118287 nel JSON).
✅ Dataset e DataLoader creati con successo (Custom Class).

Batch caricato!
Numero immagini: 4
Shape prima immagine: torch.Size([3, 427, 640])
Target keys: dict_keys(['boxes', 'labels', 'image_id', 'area', 'iscrowd'])
Box format (primo box): tensor([145.2300,  36.9700, 232.3900,  73.5400])


In [5]:
"""Esempio minimale di Train per asserire la correttezza e usabilità del dataloader."""
import torch
import torchvision
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

# --- 1. Definiamo il Modello ---
# Carichiamo il modello pre-addestrato su COCO.
# Questo modello ha 91 classi (90 oggetti + 1 background), che combaciano con gli ID del tuo JSON.
print("Caricamento modello...")
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights)

# --- 2. Setup del Device ---
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Spostiamo il modello sulla GPU
model.to(device)

# --- 3. Training Loop Minimale ---
# Mettiamo il modello in modalità train (restituisce le loss)
model.train()

# Definiamo l'optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

print("Inizio test training (solo 10 batch)...")

for i, (images, targets) in enumerate(data_loader):
    
    # Spostiamo le immagini e i target su GPU
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    # Forward Pass (Calcolo Loss)
    loss_dict = model(images, targets)
    losses = sum(loss for loss in loss_dict.values())

    # Backward Pass (Aggiornamento Pesi)
    optimizer.zero_grad()
    losses.backward()
    optimizer.step()

    # Log
    print(f"Batch {i+1} | Total Loss: {losses.item():.4f}")
    
    if i == 0:
        print("\n   Dettaglio Loss (Batch 1):")
        for k, v in loss_dict.items():
            print(f"   - {k}: {v.item():.4f}")
        print("-" * 30)

    # Stop dopo 10 batch
    if i >= 9:
        print("\n✅ Test completato! Tutto funzionante.")
        break

Caricamento modello...
Using device: cuda
Inizio test training (solo 10 batch)...
Batch 1 | Total Loss: 0.8919

   Dettaglio Loss (Batch 1):
   - loss_classifier: 0.3101
   - loss_box_reg: 0.3642
   - loss_objectness: 0.1268
   - loss_rpn_box_reg: 0.0908
------------------------------
Batch 2 | Total Loss: 0.4024
Batch 3 | Total Loss: 0.6977
Batch 4 | Total Loss: 0.6152
Batch 5 | Total Loss: 0.3278
Batch 6 | Total Loss: 0.3012
Batch 7 | Total Loss: 0.5618
Batch 8 | Total Loss: 0.4676
Batch 9 | Total Loss: 0.6221
Batch 10 | Total Loss: 0.2586

✅ Test completato! Tutto funzionante.
